In [128]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [129]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [130]:
df = pd.read_csv('ipl_till_2021_modified.csv')

In [131]:
df.head(2)

,Unnamed: 0,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [132]:
df.batting_team.unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Punjab Kings', 'Chennai Super Kings', 'Rajasthan Royals',
       'Delhi Capitals', 'Mumbai Indians', 'Sunrisers Hyderabad'],
      dtype=object)

In [133]:
# what to do?
# 1.dot %: no of dot balls/total runs*100 : should be minimum
# 2.powerplay runs: total runs in overs from 0.1-5.6 : should be maximum
# 3.death runs: total runs in overs from 17.1-20.6 : should be maximum
# 4.no of boundaries : should be maximum


# 1.Dot percentage

In [134]:
df['isDot'] = df['total_runs'].apply(lambda x: 1 if x == 0 else 0)

In [135]:
df.head(5)

,Unnamed: 0,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,isDot
0,0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0
1,1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0
2,2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,1
3,3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0
4,4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0


In [136]:
Dots = pd.DataFrame(df.groupby('batting_team')['isDot'].sum()).reset_index().rename(columns = {'isDot' : 'Dots'})

In [137]:
Dots

,batting_team,Dots
0,Chennai Super Kings,7290
1,Delhi Capitals,7458
2,Kolkata Knight Riders,7712
3,Mumbai Indians,8198
4,Punjab Kings,7534
5,Rajasthan Royals,6689
6,Royal Challengers Bangalore,7547
7,Sunrisers Hyderabad,7753


In [138]:
Total_runs = pd.DataFrame(df.groupby('batting_team')['total_runs'].sum()).reset_index()

In [139]:
Total_runs

,batting_team,total_runs
0,Chennai Super Kings,28069
1,Delhi Capitals,27864
2,Kolkata Knight Riders,28134
3,Mumbai Indians,30490
4,Punjab Kings,28702
5,Rajasthan Royals,24353
6,Royal Challengers Bangalore,28006
7,Sunrisers Hyderabad,29227


In [140]:
dot_per = pd.merge(Total_runs, Dots, on = 'batting_team')

In [141]:
dot_per

,batting_team,total_runs,Dots
0,Chennai Super Kings,28069,7290
1,Delhi Capitals,27864,7458
2,Kolkata Knight Riders,28134,7712
3,Mumbai Indians,30490,8198
4,Punjab Kings,28702,7534
5,Rajasthan Royals,24353,6689
6,Royal Challengers Bangalore,28006,7547
7,Sunrisers Hyderabad,29227,7753


In [142]:
dot_per['Dot%'] = np.round(dot_per['Dots']/dot_per['total_runs']*100,4)

In [143]:
dot_per = dot_per[['batting_team','Dot%']]

In [144]:
dot_per

,batting_team,Dot%
0,Chennai Super Kings,25.9717
1,Delhi Capitals,26.7657
2,Kolkata Knight Riders,27.4117
3,Mumbai Indians,26.8875
4,Punjab Kings,26.2490
5,Rajasthan Royals,27.4668
6,Royal Challengers Bangalore,26.9478
7,Sunrisers Hyderabad,26.5268


In [145]:
# point 1 done

# 2.Powerplay runs

In [146]:
df.over.unique()

array([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  0,  1,  2,
        3,  4,  5], dtype=int64)

In [147]:
def getPhase(over):
    if over < 6:
        return 'Powerplay'
    elif over < 16:
        return 'Middle'
    else:
        return 'Death'

In [148]:
df['Phase'] = df['over'].apply(lambda x: getPhase(x))

In [149]:
df.head(2)

,Unnamed: 0,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,isDot,Phase
0,0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0,Middle
1,1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,0,Middle


In [150]:
def getPhasedf(df, current_phase):
    df = df[df.Phase == current_phase]
    df.reset_index(inplace = True, drop = True)
    
    df = pd.DataFrame(df.groupby('batting_team')['total_runs'].sum()).reset_index()
    
    return df

In [151]:
powerplay_df = getPhasedf(df,'Powerplay')

In [152]:
powerplay_df = powerplay_df.rename(columns = {'total_runs':'powerplay_runs'})

In [153]:
powerplay_df

,batting_team,powerplay_runs
0,Chennai Super Kings,7956
1,Delhi Capitals,8390
2,Kolkata Knight Riders,8412
3,Mumbai Indians,8677
4,Punjab Kings,8470
5,Rajasthan Royals,7068
6,Royal Challengers Bangalore,8147
7,Sunrisers Hyderabad,8903


In [154]:
# point 2 done

# 3.Death overs runs

In [155]:
death_df = getPhasedf(df, 'Death')

In [156]:
death_df = death_df.rename(columns = {'total_runs':'death_runs'})

In [157]:
death_df

,batting_team,death_runs
0,Chennai Super Kings,6598
1,Delhi Capitals,5816
2,Kolkata Knight Riders,5975
3,Mumbai Indians,7037
4,Punjab Kings,5970
5,Rajasthan Royals,5065
6,Royal Challengers Bangalore,6194
7,Sunrisers Hyderabad,6338


In [158]:
# point 3 done

# 4.No. of boundaries

In [159]:
# df

In [160]:
df['isSix'] = df['total_runs'].apply(lambda x: 1 if x == 6 else 0)
df['isFour'] = df['total_runs'].apply(lambda x: 1 if x == 4 else 0)

In [161]:
# df.head(20)

In [162]:
df['isBoundary'] = df['isSix'] + df['isFour'] 

In [163]:
# df.head(30)

In [164]:
Boundaries = pd.DataFrame(df.groupby('batting_team')['isBoundary'].sum()).reset_index().rename(columns = {'isBoundary' : 'Boundaries'})

In [165]:
Boundaries

,batting_team,Boundaries
0,Chennai Super Kings,3439
1,Delhi Capitals,3416
2,Kolkata Knight Riders,3540
3,Mumbai Indians,3851
4,Punjab Kings,3583
5,Rajasthan Royals,2984
6,Royal Challengers Bangalore,3465
7,Sunrisers Hyderabad,3454


In [166]:
# point 4 done

In [167]:
# getting required data
req_df = pd.merge(dot_per, powerplay_df, on = 'batting_team').merge(death_df, on = 'batting_team').merge(Boundaries, on = 'batting_team')

In [168]:
req_df

,batting_team,Dot%,powerplay_runs,death_runs,Boundaries
0,Chennai Super Kings,25.9717,7956,6598,3439
1,Delhi Capitals,26.7657,8390,5816,3416
2,Kolkata Knight Riders,27.4117,8412,5975,3540
3,Mumbai Indians,26.8875,8677,7037,3851
4,Punjab Kings,26.2490,8470,5970,3583
5,Rajasthan Royals,27.4668,7068,5065,2984
6,Royal Challengers Bangalore,26.9478,8147,6194,3465
7,Sunrisers Hyderabad,26.5268,8903,6338,3454


# Performing MCDM

In [169]:
# considering random weights to above criteria:
# giving 80-20 weightage to benefit and loss criteria
wt_dot, wt_powerplay, wt_death, wt_boundaries = 0.2, 0.26, 0.26, 0.26

In [170]:
def Topsis(df):
    # step 1: Calculate normalized matrix and then weighted normalized matrix
    # calculating squares of each column
    df['calcDot%'] = df['Dot%'].apply(lambda x: x*x)
    df['calcpowerplay_runs'] = df['powerplay_runs'].apply(lambda x: x*x)
    df['calcdeath_runs'] = df['death_runs'].apply(lambda x: x*x)
    df['calcBoundaries'] = df['Boundaries'].apply(lambda x: x*x)
    
    sq_dot, sq_powerplay, sq_death, sq_boundaries = np.sqrt(df[['calcDot%','calcpowerplay_runs','calcdeath_runs','calcBoundaries']]).sum(axis = 0)
    
    # calculating normalized matrix
    df['calcDot%'] = df['calcDot%'].apply(lambda x: x/sq_dot)
    df['calcpowerplay_runs'] = df['calcpowerplay_runs'].apply(lambda x: x/sq_powerplay)
    df['calcdeath_runs'] = df['calcdeath_runs'].apply(lambda x: x/sq_death)
    df['calcBoundaries'] = df['calcBoundaries'].apply(lambda x: x/sq_boundaries)
    
     # calculating weighted normalized matrix by multiplying by weights to each value in column
    df['calcDot%'] = df['calcDot%'].apply(lambda x: x*wt_dot)
    df['calcpowerplay_runs'] = df['calcpowerplay_runs'].apply(lambda x: x*wt_powerplay)
    df['calcdeath_runs'] = df['calcdeath_runs'].apply(lambda x: x*wt_death)
    df['calcBoundaries'] = df['calcBoundaries'].apply(lambda x: x*wt_boundaries)
    
    # calculating best and worst value columnwise which is max and min value respectively for benefit criteria and min and max value for loss criteria
    best_dot, worst_dot = min(df['calcDot%']), max(df['calcDot%'])
    best_powerplay, worst_powerplay = max(df['calcpowerplay_runs']), min(df['calcpowerplay_runs'])
    best_death, worst_death = max(df['calcdeath_runs']), min(df['calcdeath_runs'])
    best_boundaries, worst_boundaries = max(df['calcBoundaries']), min(df['calcBoundaries'])
    
     # step 2: To calculate ideal best and ideal worst
    df['best_dot1'] = df['calcDot%'].apply(lambda x: (x-best_dot)*(x-best_dot))
    df['best_powerplay1'] = df['calcpowerplay_runs'].apply(lambda x: (x-best_powerplay)*(x-best_powerplay))
    df['best_death1'] = df['calcdeath_runs'].apply(lambda x: (x-best_death)*(x-best_death))
    df['best_boundaries1'] = df['calcBoundaries'].apply(lambda x: (x-best_boundaries)*(x-best_boundaries))
    
    df['best_sqrt'] = df.apply(lambda x: x['best_dot1']+x['best_powerplay1']+x['best_death1']+x['best_boundaries1'], axis = 1)
    
    df['worst_dot1'] = df['calcDot%'].apply(lambda x: (x-worst_dot)*(x-worst_dot))
    df['worst_powerplay1'] = df['calcpowerplay_runs'].apply(lambda x: (x-worst_powerplay)*(x-worst_powerplay))
    df['worst_death1'] = df['calcdeath_runs'].apply(lambda x: (x-worst_death)*(x-worst_death))
    df['worst_boundaries1'] = df['calcBoundaries'].apply(lambda x: (x-worst_boundaries)*(x-worst_boundaries))
    
    df['worst_sqrt'] = df.apply(lambda x: x['worst_dot1']+x['worst_powerplay1']+x['worst_death1']+x['worst_boundaries1'], axis = 1)
    
    # step 3: Find TOPSIS score 
    df['score'] = df.apply(lambda x: x['worst_sqrt']/(x['worst_sqrt']+x['best_sqrt']), axis = 1)
    
    return df

In [171]:
newdf = Topsis(req_df)

In [172]:
newdf = newdf.sort_values(by = 'score', ascending = False).reset_index()
newdf = newdf[['batting_team','powerplay_runs','death_runs','Boundaries','Dot%','score']]

In [173]:
newdf

,batting_team,powerplay_runs,death_runs,Boundaries,Dot%,score
0,Mumbai Indians,8677,7037,3851,26.8875,0.991652
1,Sunrisers Hyderabad,8903,6338,3454,26.5268,0.862385
2,Chennai Super Kings,7956,6598,3439,25.9717,0.684987
3,Punjab Kings,8470,5970,3583,26.2490,0.634145
4,Kolkata Knight Riders,8412,5975,3540,27.4117,0.606039
5,Royal Challengers Bangalore,8147,6194,3465,26.9478,0.585430
6,Delhi Capitals,8390,5816,3416,26.7657,0.499828
7,Rajasthan Royals,7068,5065,2984,27.4668,0.000000
